In [1]:
%matplotlib inline
""" handling files support packages """
from glob import glob

""" logic support packages """
import numpy as np
import pandas as pd

""" plot support packages """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image
#import pydotplus

""" image trasformation packages """
from PIL import Image
import skimage.io as skio

""" statistical data visualization packages"""
import seaborn as sns

""" machine learning functions """
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

""" seaborn configurations """
sns.set_style('white')
sns.set_context('talk')
plt.rcParams['figure.figsize'] = 20, 10

In [2]:
from funcoes import *

In [3]:
#letras, letters_dict = ler_letras("../letras.csv")

In [4]:
data = pd.read_csv("../letras.csv", header = 0, sep=",")
list(data.columns.values)

['path', 'rotulo', 'caixa_alta_baixa']

In [5]:
#data['rotulo']

In [6]:
img_pixels = Image.open(data['path'][0])

In [7]:
#for i, file in enumerate(list(data['path'])):
    #print(file)

In [8]:
X = []
y = []

for i, file in enumerate(list(data['path'])):
    label = data['rotulo'][i]
    img_pixels = list(Image.open(file).getdata())
    X.append(img_pixels)
    y.append(label)

In [9]:
#from sklearn import preprocessing

#le = preprocessing.LabelEncoder()
#le.fit(y)
#ley = le.transform(y)

# Unsupervised Learnind: Dimensionality Reduction

In [10]:
print("temos {0} imagens cada uma com {1} dimensoes".format(len(X), len(X[0])))

temos 720 imagens cada uma com 1750 dimensoes


In [11]:
#from sklearn.manifold import Isomap

In [12]:
#iso = Isomap(n_components = 2)
#data_projected = iso.fit_transform(X)

In [13]:
#data_projected.shape

In [14]:
#plt.scatter(data_projected[:, 0], data_projected[:, 1], c=ley,
#            edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('nipy_spectral'))
#plt.colorbar(label='digita label', ticks=range(10))
#plt.clim(-0.5, 9.5)

A base relativa aos pixels esta muito confusa, as classes estao muito misturadas para os features (imagens).

# Divisao da base em treino e teste e aplicacao de algoritmos de aprendizado

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [16]:
%%time
from sklearn.neural_network import MLPClassifier

MLPparams = {'hidden_layer_sizes':(100, 20, 50),
             'activation' : ('identity', 'logistic', 'tanh', 'relu'),
             'solver' : ('lbfgs', 'sgd', 'adam'),
             'learning_rate': ('constant', 'invscaling', 'adaptative')
            }

svr = MLPClassifier(max_iter = 200000)
clfMLP = GridSearchCV(svr, MLPparams)
clfMLP = MLPClassifier()
clfMLP.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfMLP.score(X_test, y_test)) )
print( "Best parameters: " + str(clfMLP.get_params) )

Score: 0.013888888888888888
Best parameters: <bound method BaseEstimator.get_params of MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)>
CPU times: user 6.95 s, sys: 7.62 s, total: 14.6 s
Wall time: 3.97 s


In [17]:
%%time
from sklearn.svm import SVC
clf = SVC() # kernel = 'poly', degree = 2, max_iter = 100000

SVMparams = { 'kernel' : ('linear', 'poly', 'rbf', 'sigmoid')}

svr = SVC()
clf = GridSearchCV(svr, SVMparams)

clf.fit(X_train, y_train)
print( "Score: {0}".format(clf.score(X_test, y_test)) )
print( "Best parameters: " + str(clf.best_params_) )

Score: 0.33796296296296297
Best parameters: {'kernel': 'linear'}
CPU times: user 12.1 s, sys: 180 ms, total: 12.2 s
Wall time: 12.1 s


In [18]:
%%time
from sklearn.svm import SVC
clf = SVC(kernel = 'poly')

clf.fit(X_train, y_train)
print( "Score: {0}".format(clf.score(X_test, y_test)) )

Score: 0.3472222222222222
CPU times: user 1.38 s, sys: 0 ns, total: 1.38 s
Wall time: 1.38 s


In [19]:
%%time
from sklearn.ensemble import RandomForestClassifier
clfRF_balanced = RandomForestClassifier()


RFparams = {'n_estimators' : (10, 2, 3, 5, 7),
           'criterion' : ('gini', 'entropy'),
           'min_samples_split' : (2, 10, 40),
           'min_samples_leaf' : (1, 5, 10, 40)}

svr = RandomForestClassifier()
clfRF_balanced = GridSearchCV(svr, RFparams)
clfRF_balanced.fit( X = X_train, y = y_train )
print( "Score: {0}".format(clfRF_balanced.score(X_test, y_test)) )
print( "Best parameters: " + str(clfRF_balanced.best_params_) )

Score: 0.17592592592592593
Best parameters: {'criterion': 'entropy', 'n_estimators': 10, 'min_samples_split': 10, 'min_samples_leaf': 1}
CPU times: user 33 s, sys: 36 ms, total: 33 s
Wall time: 33 s


In [20]:
#imgs = skio.imread_collection(list(data['path']))
#imgA = crop_char(imgs[0], 0)

In [21]:
#print(clf.predict([item for sublist in imgA.tolist() for item in sublist]))

In [22]:
#joblib.dump(clf, 'classifier_image.pkl') 